In [1]:
import numpy as np
import scipy.signal
import matplotlib.pyplot as plt
import wavio
import IPython

In [2]:
def single_channel_fxlms(un, usn, dn, M, mu):
    """
    :param un: unknow signal
    :type  un: numpy.ndarray
    
    :param usn: unknow signal with secondary path
    :type  usn: numpy.ndarray
    
    :param dn: expectation signal
    :type  dn: numpy.ndarray
    
    :param M : length of FIR filter
    :type  M : int
    
    :param mu: learning rate
    :type  mu: float
    
    :return: yn, en
    """
    # To ensure the length of un and dn is same
    ITER = min(len(un),len(dn), len(usn)) 
    
    # Signal buffer, reference input to FIR filter
    u = np.zeros(M)
    
    # Secondary buffer, reference input to LMS algorithm
    us = np.zeros(M)
    
    # FIR Weights
    w = np.zeros(M)
    
    # Estimated Signal, FIR Product
    yn = np.zeros(ITER) 
    
    # Estimated Signal consider secondary path, FIR Product
    sn = np.zeros(ITER)
    
    # Error Signal, difference between estimation and expectation
    en = np.zeros(ITER) 
    
    for n in range(ITER):
        u[1:M] = u[0:M-1]
        u[0] = un[n]
        
        us[1:M] = us[0:M-1]
        us[0] = usn[n]
        
        yn[n] = np.dot(w.T, u)
        sn[n] = np.dot(w.T, us)
        
        en[n] = dn[n] - sn[n]
        
        w = w + mu * en[n] * us
    return sn, en

In [6]:
music_raw = wavio.read("16k/mix.wav")
room_drum_raw = wavio.read("16k/room_drum.wav")
drum_raw = wavio.read("16k/drum.wav")

music = music_raw.data[:, 0]
music = music.astype(np.float32, order='C') / 32767.0

room_drum = room_drum_raw.data[:, 0]
room_drum = room_drum.astype(np.float32, order='C') / 32767.0

drum = drum_raw.data[:, 0]
drum = drum.astype(np.float32, order='C') / 32767.0

In [7]:
yn, en = single_channel_fxlms(drum, room_drum, music, 1, 0.25)

In [9]:
sr = 16000
wavio.write("result/en_single_fxlms.wav", en, sr, sampwidth=3)
wavio.write("result/yn_single_fxlms.wav", yn, sr, sampwidth=3)
IPython.display.display(IPython.display.Audio("result/en_single_fxlms.wav", rate=sr))
IPython.display.display(IPython.display.Audio("result/yn_single_fxlms.wav", rate=sr))
IPython.display.display(IPython.display.Audio("16k/mix.wav", rate=sr))